In [1]:
!pip uninstall -y flwr
!pip install flwr==1.23.0

# !pip install -q flwr[simulation]
!pip install torch
!pip install torchvision
!pip install matplotlib
!pip install kaggle

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 732.0/732.0 KB 4.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 KB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 KB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 770.3/770.3 KB 4.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 5.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 KB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 5.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 KB 4.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 5.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 5.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.4/242.4 KB 3.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
import os

os.environ["FLWR_SIMULATION_USE_RAY"] = "0"
os.environ["FLWR_LOGGING"] = "error"

import random
from typing import List
from collections import OrderedDict

import numpy as np
import pandas as pd
from PIL import Image

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.model_selection import GroupShuffleSplit, StratifiedGroupKFold
import torchvision.transforms as transforms
import torchvision.models as models
import matplotlib.pyplot as plt
import seaborn as sns
import flwr as fl
# from flwr.common import Context
import cv2

from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, roc_auc_score, confusion_matrix, classification_report
)

# (ADJUSTMENTS) NOVAS BIBLIOTECAS:
from torchvision.models import ResNet18_Weights
import torch.nn.functional as F


In [ ]:
print(fl.__version__)

# Configs

In [ ]:
NUM_CLIENTS = 7
RANDOM_SEED = 42
BATCH_SIZE = 64
ROUNDS = 22
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
IMG_SIZE = 224 # Imagenet size (224x224)
TEST_SIZE = 0.15
VALID_SIZE = 0.15
DATA_TRANSFORM = {
    "img_size": IMG_SIZE,
    "normalizer": { # Normalizer for Imagenet
        "mean":[0.485, 0.456, 0.406],
        "std":[0.229, 0.224, 0.225]
    }
}

MODEL_CONFIG = {
    "epochs": 3,
    "batch_size": 64,
    "learning_rate": 0.001,
    'criterion': nn.CrossEntropyLoss(),
    'optimizer': torch.optim.Adam,
    'patience': 3
}

In [ ]:
# (ADJUSTMENTS) REMOVENDO AS CONFIGURAÇÕES DO GOOGLE DRIVE, PARA EXECUÇÃO LOCAL.
'''
    from google.colab import drive
    drive.mount('/content/drive')

    os.chdir("/content/drive/MyDrive/MO839-FL")
'''

In [ ]:
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

# Dataset

In [ ]:
kidney_dataset = pd.read_csv('G:\\Meu Drive\\Materias\\MO839-FL\\trabalho_pratico\\simulações_scripts\\data\\kidneyData.csv')
kidney_dataset

In [ ]:
LABELS = sorted(kidney_dataset['Class'].unique())
LABEL2TARGET = {str(l): i for i, l in enumerate(LABELS)}
NUM_CLASSES = len(LABELS)

In [ ]:
LABEL2TARGET

In [ ]:
# Focal Loss:
from sklearn.utils.class_weight import compute_class_weight

class_weights = compute_class_weight(
    class_weight="balanced",
    classes=np.unique(kidney_dataset["Class"]),
    y=kidney_dataset["Class"]
)

class_weights = torch.tensor(class_weights, dtype=torch.float).to(DEVICE)


MODEL_CONFIG['criterion'] = nn.CrossEntropyLoss(weight=class_weights, label_smoothing=0.05)

In [ ]:
class KidneyData(Dataset):
    def __init__(self, metadata: pd.DataFrame, label_map: dict, transform=None):
        self.metadata = metadata.reset_index(drop=True)
        self.transform = transform
        self.label_map = label_map

    def __len__(self):
        return len(self.metadata)

    def __getitem__(self, idx):
        row = self.metadata.iloc[idx]
        img_path = row['path']
        label_str = str(row['Class'])
        label = int(self.label_map[label_str])
        img = Image.open(img_path).convert("RGB")
        if self.transform:
            img = self.transform(img)
        return img, torch.tensor(label, dtype=torch.long)


In [ ]:
classes = kidney_dataset['Class'].unique()

fig, ax = plt.subplots(1, len(classes), figsize=(12, 12))
for i, c in enumerate(classes):
    class_df = kidney_dataset[kidney_dataset['Class'] == c]
    random_integer = random.randint(1, 10)
    data = KidneyData(class_df, label_map=LABEL2TARGET)
    img, label = data.__getitem__(random.randint(1, len(class_df)))
    print(f"image {c} format {np.array(img).shape}")
    img = cv2.cvtColor(np.array(img), cv2.COLOR_BGR2RGB)
    plt.subplot(1, len(classes), i+1)
    plt.imshow(img)
    plt.title(c)
    plt.axis('off')
plt.show()

## Splitting data

### Separating some data for testing global model and get clients data

In [ ]:
# Preserving classes distribution on test set
splitter = StratifiedGroupKFold(n_splits=int(1/TEST_SIZE), shuffle=True, random_state=RANDOM_SEED)
clients_idx, test_idx = next(splitter.split(
    kidney_dataset,
    kidney_dataset["Class"],
    groups=kidney_dataset["group"]
    )
)

clients_df = kidney_dataset.iloc[clients_idx].reset_index(drop=True)
test_df  = kidney_dataset.iloc[test_idx].reset_index(drop=True)


print("Clients groups:", clients_df["group"].nunique())
print("Test groups:", test_df["group"].nunique())
print("Intersection:", set(clients_df["group"]) & set(test_df["group"]))

plt.figure(figsize=(8, 4))

sns.countplot(x=test_df['Class'])
plt.title("Data available to server only for test model")

plt.tight_layout()
plt.show()

### Get clients (not available data for server)

In [ ]:
def split_on_clients(clients_df: pd.DataFrame, num_clients: int = NUM_CLIENTS, val_frac: float = VALID_SIZE):
    """
    Splits data in NUM_CLIENTS clients, randomly, splitting in train and validation data inside each client (like small datasets)

    """
    unique_groups = clients_df["group"].unique()
    np.random.seed(42)
    np.random.shuffle(unique_groups)
    group_splits = np.array_split(unique_groups, num_clients)

    clients = {}

    for i, split in enumerate(group_splits):
        client_df = clients_df[clients_df["group"].isin(split)]
        splitter = GroupShuffleSplit(n_splits=1, test_size = val_frac, random_state=RANDOM_SEED)
        train_idx, val_idx = next(splitter.split(client_df, client_df["Class"], groups=client_df["group"]))

        client_train_df = client_df.iloc[train_idx].reset_index(drop=True)
        client_val_df = client_df.iloc[val_idx].reset_index(drop=True)

        clients[i] = {'train': client_train_df, 'valid': client_val_df}

    return clients

# def split_on_clients(clients_df, num_clients=NUM_CLIENTS, val_frac=VALID_SIZE):
#     groups = list(clients_df["group"].unique())
#     groups.sort()

#     # calcula tamanho de cada grupo
#     group_sizes = {g: len(clients_df[clients_df["group"] == g]) for g in groups}

#     # ordena grupos do maior para menor
#     groups_sorted = sorted(groups, key=lambda g: group_sizes[g], reverse=True)

#     # distribui grupos balanceando o total de imagens por cliente
#     client_buckets = [[] for _ in range(num_clients)]
#     bucket_sizes = [0] * num_clients

#     for g in groups_sorted:
#         smallest = bucket_sizes.index(min(bucket_sizes))
#         client_buckets[smallest].append(g)
#         bucket_sizes[smallest] += group_sizes[g]

#     clients = {}
#     for cid in range(num_clients):
#         cid_df = clients_df[clients_df["group"].isin(client_buckets[cid])]

#         splitter = GroupShuffleSplit(n_splits=1, test_size=val_frac, random_state=RANDOM_SEED)
#         train_idx, val_idx = next(splitter.split(cid_df, cid_df["Class"], groups=cid_df["group"]))

#         clients[cid] = {
#             "train": cid_df.iloc[train_idx].reset_index(drop=True),
#             "valid": cid_df.iloc[val_idx].reset_index(drop=True)
#         }

#     return clients


In [ ]:
# (ADJUSTMENTS)
def split_on_non_iid_clients(clients_df: pd.DataFrame,
                             num_clients: int = NUM_CLIENTS,
                             val_frac: float = VALID_SIZE,
                             alpha: float = 0.5):
    """
    Non-IID usando distribuição Dirichlet.
    - Garante todas as classes em todos os clientes
    - Mantém heterogeneidade controlada
    - Evita clientes com classes faltando
    """
    
    np.random.seed(42)
    clients = {}

    # Converte labels para numpy
    labels = clients_df["Class"].values
    unique_classes = np.unique(labels)

    # Índices do dataframe completo
    all_indices = np.arange(len(clients_df))

    # Lista de índices para cada cliente
    client_indices = [[] for _ in range(num_clients)]

    # Para cada classe, divide seus índices entre os clientes
    for cls in unique_classes:
        cls_idx = all_indices[labels == cls]

        # distribuição Dirichlet (controla Non-IID)
        proportions = np.random.dirichlet(alpha=[alpha] * num_clients)
        proportions = (proportions * len(cls_idx)).astype(int)

        start = 0
        for cid, amount in enumerate(proportions):
            client_indices[cid].extend(cls_idx[start:start + amount])
            start += amount

    # Embaralha
    for cid in range(num_clients):
        np.random.shuffle(client_indices[cid])

        cid_df = clients_df.iloc[client_indices[cid]].reset_index(drop=True)

        # split train/validation por cliente
        splitter = GroupShuffleSplit(n_splits=1, test_size=val_frac, random_state=42)
        train_idx, val_idx = next(splitter.split(cid_df, cid_df["Class"], groups=cid_df["group"]))

        client_train_df = cid_df.iloc[train_idx].reset_index(drop=True)
        client_val_df = cid_df.iloc[val_idx].reset_index(drop=True)

        clients[cid] = {'train': client_train_df, 'valid': client_val_df}

    return clients


In [ ]:
random_clients = split_on_clients(clients_df, NUM_CLIENTS, VALID_SIZE)

fig, axes = plt.subplots(2, NUM_CLIENTS, figsize=(22, 8), sharey=True)
axes = axes.flatten()

for i, (client_id, data) in enumerate(random_clients.items()):
    # Plot Train
    sns.countplot(x=data["train"]["Class"], ax=axes[i])
    axes[i].set_title(f"Client {client_id} - Train")
    axes[i].set_xlabel("")
    axes[i].set_ylabel("")

    # Plot Validation
    sns.countplot(x=data["valid"]["Class"], ax=axes[i + NUM_CLIENTS])
    axes[i + NUM_CLIENTS].set_title(f"Client {client_id} - Val")
    axes[i + NUM_CLIENTS].set_xlabel("")
    axes[i + NUM_CLIENTS].set_ylabel("")

plt.tight_layout()
plt.show()

In [ ]:
non_iid_clients = split_on_non_iid_clients(clients_df, NUM_CLIENTS, VALID_SIZE, alpha=1.0)

fig, axes = plt.subplots(2, NUM_CLIENTS, figsize=(22, 8), sharey=True)
axes = axes.flatten()

for i, (client_id, data) in enumerate(non_iid_clients.items()):
    # Plot Train
    sns.countplot(x=data["train"]["Class"], ax=axes[i])
    axes[i].set_title(f"Client {client_id} - Train")
    axes[i].set_xlabel("")
    axes[i].set_ylabel("")

    # Plot Validation
    sns.countplot(x=data["valid"]["Class"], ax=axes[i + NUM_CLIENTS])
    axes[i + NUM_CLIENTS].set_title(f"Client {client_id} - Val")
    axes[i + NUM_CLIENTS].set_xlabel("")
    axes[i + NUM_CLIENTS].set_ylabel("")

plt.tight_layout()
plt.show()

## Loading data

In [ ]:
# (ADJUSTMENTS) 
'''
    Altera a configuração da variável `transform`, para:
    - Reduzir o overfitting local
    - Melhorar a generalização
    - Seguir os padrões do ResNet Pré-Treinado
'''


def load_datasets(
    clients: dict,
    test_df: pd.DataFrame,
    transforms_config: dict,
    batch_size: int = 32,
    label_map = dict
):
    img_size = transforms_config['img_size']
    normalize_config = transforms_config['normalizer']
    # transform = transforms.Compose([
    #     transforms.Resize((224, 224)),
    #     transforms.RandomHorizontalFlip(),
    #     transforms.RandomRotation(10),
    #     transforms.ColorJitter(
    #         brightness=0.2,
    #         contrast=0.2,
    #         saturation=0.2,
    #         hue=0.02
    #     ),
    #     transforms.RandomAdjustSharpness(sharpness_factor=1.5, p=0.3),
    #     transforms.ToTensor(),
    #     transforms.Normalize(
    #         [0.485, 0.456, 0.406],
    #         [0.229, 0.224, 0.225]
    #     ),
    # ])
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(8),
        transforms.ToTensor(),
        transforms.Normalize(
            [0.485, 0.456, 0.406],
            [0.229, 0.224, 0.225]
        ),
    ])
        
    trainloaders = []
    valloaders = []

    for client_id, data in clients.items():
        train_dataset = KidneyData(data["train"], label_map, transform)
        val_dataset = KidneyData(data["valid"], label_map, transform)

        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
        val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

        trainloaders.append(train_loader)
        valloaders.append(val_loader)

    test_dataset = KidneyData(test_df, label_map, transform)
    testloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

    return trainloaders, valloaders, testloader

In [ ]:
########## 
random_clients = split_on_clients(clients_df, NUM_CLIENTS, VALID_SIZE)

trainloaders, valloaders, testloader = load_datasets(
    clients=non_iid_clients,
    test_df=test_df,
    transforms_config = DATA_TRANSFORM,
    batch_size = BATCH_SIZE,
    label_map=LABEL2TARGET
)

# Model

In [ ]:
import copy

def train(model,
          trainloader,
          valloader,
          device,
          patience,
          global_params=None,
          mu=0.01,
          verbose=False):

    epochs = MODEL_CONFIG['epochs']
    criterion = MODEL_CONFIG['criterion']
    optimizer_class = MODEL_CONFIG['optimizer']
    lr = MODEL_CONFIG['learning_rate']

    # Weight decay + only trainable params
    optimizer = optimizer_class(
        filter(lambda p: p.requires_grad, model.parameters()),
        lr=lr,
        weight_decay=1e-4
    )

    # Scheduler para estabilizar FL
    scheduler = torch.optim.lr_scheduler.StepLR(
        optimizer,
        step_size=2,
        gamma=0.5
    )

    model.to(device)

    best_val_loss = float("inf")
    epochs_no_improve = 0
    best_state_dict = None

    for epoch in range(epochs):

        model.train()
        correct, total = 0, 0
        epoch_loss = 0.0

        for images, labels in trainloader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()

            outputs = model(images)
            loss = criterion(outputs, labels)

            # -----------------------------------
            # FEDPROX — TERMO PROXIMAL REAL
            # -----------------------------------
            if global_params is not None:
                prox = 0.0
                for w, w_global in zip(model.parameters(), global_params):
                    prox += ((w - w_global.to(device)) ** 2).sum()
                loss = loss + (mu / 2) * prox
            # -----------------------------------

            loss.backward()

            # Clipping para estabilizar FL
            torch.nn.utils.clip_grad_norm_(model.parameters(), 5.0)

            optimizer.step()

            epoch_loss += loss.item() * images.size(0)
            total += labels.size(0)
            correct += (torch.argmax(outputs, dim=1) == labels).sum().item()

        # Final do epoch
        epoch_loss /= len(trainloader.dataset)
        epoch_acc = correct / total if total > 0 else 0.0

        # Step do scheduler
        scheduler.step()

        # Avaliação no cliente
        val_metrics, _ = evaluate(model, valloader, device, num_classes=NUM_CLASSES)
        val_loss = val_metrics['loss']

        # Early stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            epochs_no_improve = 0
            best_state_dict = copy.deepcopy(model.state_dict())
        else:
            epochs_no_improve += 1
            if epochs_no_improve >= patience:
                if verbose:
                    print(f"Early stopping at epoch {epoch+1}")
                break

        if verbose:
            print(f"Epoch {epoch+1}: train loss {epoch_loss:.4f}, acc {epoch_acc:.4f}, val_loss {val_loss:.4f}")

    # Retorna o melhor estado
    if best_state_dict is not None:
        model.load_state_dict(best_state_dict)

    return model



def evaluate(model, testloader, device, num_classes):
    model.to(device)
    criterion = MODEL_CONFIG['criterion']
    model.eval()

    total, correct = 0, 0
    running_loss = 0.0

    y_true, y_pred, y_prob = [], [], []

    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(device), labels.to(device, dtype=torch.long)

            outputs = model(images)
            loss = criterion(outputs, labels)

            # loss ponderado pelo tamanho do batch
            running_loss += loss.item() * labels.size(0)

            probs = torch.softmax(outputs, dim=1)
            preds = torch.argmax(probs, dim=1)

            correct += (preds == labels).sum().item()
            total += labels.size(0)

            y_true.extend(labels.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())
            y_prob.append(probs.cpu().numpy())

    # loss médio real
    loss = running_loss / total if total > 0 else 0.0
    accuracy = correct / total if total > 0 else 0.0

    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    y_prob = np.concatenate(y_prob, axis=0)

    # AUC CORRIGIDO
    try:
        if num_classes == 2:
            auc = roc_auc_score(y_true, y_prob[:, 1])
        else:
            auc = roc_auc_score(y_true, y_prob, multi_class='ovr')
    except Exception:
        auc = float('nan')

    return {
        'loss': float(loss),
        'accuracy': float(accuracy),
        'precision': float(precision_score(y_true, y_pred, average='weighted', zero_division=0)),
        'recall':  float(recall_score(y_true, y_pred, average='weighted', zero_division=0)),
        'f1-score': float(f1_score(y_true, y_pred, average='weighted', zero_division=0)),
        'auc': auc
    }, confusion_matrix(y_true, y_pred)



def get_parameters(model: nn.Module) -> List[np.ndarray]:
    return [val.detach().cpu().numpy() for _, val in model.state_dict().items()]

def set_parameters(model: nn.Module, parameters: List[np.ndarray]):
    state_dict_keys = list(model.state_dict().keys())
    new_state = OrderedDict()
    for k, arr in zip(state_dict_keys, parameters):
        ref = model.state_dict()[k]
        t = torch.tensor(arr, dtype=ref.dtype, device=ref.device)
        new_state[k] = t
    model.load_state_dict(new_state, strict=True)

## ResNet18

- Here a ResNet18, pretrained with Imagenet dataset will be used, freezing net parameter and fine tuning final layer parameters (transfer learning)

In [ ]:
# original model
model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
model

In [ ]:
# (ADJUSTMENTS) 
'''
    Pretrained ImageNet melhora MUITO classes difíceis (“Stone” e “Tumor”)
    Libera o  `layer4` permitindo o Federated Learning aprender padrões específicos do dataset renal
'''

# def build_model(num_classes):
#     with torch.device("cpu"):
#         model = models.resnet18(weights=ResNet18_Weights.DEFAULT)

#         # Congela tudo
#         for name, param in model.named_parameters():
#             param.requires_grad = False

#         # Descongela só o final da rede
#         for name, param in model.layer4.named_parameters():
#             param.requires_grad = True

#         model.fc = nn.Sequential(
#             nn.Linear(model.fc.in_features, 128),
#             nn.ReLU(),
#             nn.Dropout(0.2),
#             nn.Linear(128, num_classes)
#         )

#     return model.cpu()

# def build_model(num_classes):
#     model = models.resnet18(weights=ResNet18_Weights.DEFAULT)

#     # Congela tudo
#     for name, param in model.named_parameters():
#         param.requires_grad = False

#     # Descongela só o final da rede
#     for name, param in model.layer4.named_parameters():
#         param.requires_grad = True

#     model.fc = nn.Sequential(
#         nn.Linear(model.fc.in_features, 128),
#         nn.ReLU(),
#         nn.Dropout(0.2),
#         nn.Linear(128, num_classes)
#     )
#     return model

# from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights

# def build_model(num_classes):
#     model = efficientnet_b0(weights=EfficientNet_B0_Weights.IMAGENET1K_V1)

#     # Congela tudo exceto classifier
#     for name, param in model.named_parameters():
#         param.requires_grad = False

#     model.classifier[1] = nn.Sequential(
#         nn.Dropout(0.3),
#         nn.Linear(model.classifier[1].in_features, 256),
#         nn.ReLU(),
#         nn.Linear(256, num_classes)
#     )

#     return model

# from torchvision.models import mobilenet_v3_small, MobileNet_V3_Small_Weights

# def build_model(num_classes):
#     model = mobilenet_v3_small(weights=MobileNet_V3_Small_Weights.DEFAULT)

#     # congela tudo exceto classifier
#     for name, param in model.named_parameters():
#         if "classifier" not in name:
#             param.requires_grad = False

#     model.classifier = nn.Sequential(
#         nn.Linear(model.classifier[0].in_features, 256),
#         nn.ReLU(),
#         nn.Dropout(0.2),
#         nn.Linear(256, num_classes)
#     )
#     return model

# def build_model(num_classes):
#     model = models.resnet18(weights=ResNet18_Weights.DEFAULT)

#     # CONGELA TUDO MENOS layer4
#     for name, param in model.named_parameters():
#         if not (("layer4" in name) or ("layer3" in name)):
#             param.requires_grad = False

#     model.fc = nn.Sequential(
#         nn.Linear(model.fc.in_features, 256),
#         nn.ReLU(),
#         nn.Dropout(0.3),
#         nn.Linear(256, num_classes)
#     )
#     return model

def build_model(num_classes):
    model = models.resnet18(weights=ResNet18_Weights.DEFAULT)

    for name, param in model.named_parameters():
        param.requires_grad = False   # <-- congela TUDO

    # só treina a última camada
    model.fc = nn.Linear(model.fc.in_features, num_classes)
    return model


- Adapted model

In [ ]:
model = build_model(num_classes = NUM_CLASSES)
model

In [ ]:
# Check trainable and not trainable layers
trainable_layers = []
frozen_layers = []

for name, param in model.named_parameters():
    if param.requires_grad:
        trainable_layers.append(name)
    else:
        frozen_layers.append(name)

print("Trainable layers:")
for name in trainable_layers:
    print(name)

print("\nFrozen layers:")
for name in frozen_layers:
    print(name)

# Flower and Federated Learning

## Client

In [ ]:
# (ADJUSTMENTS)
BASE_MODEL = build_model(NUM_CLASSES)
BASE_MODEL = BASE_MODEL.to("cpu")             # <-- IMPORTANTE
BASE_STATE = {k: v.cpu() for k, v in BASE_MODEL.state_dict().items()}

class KidneyClient(fl.client.NumPyClient):
    def __init__(
        self,
        model,
        trainloader,
        valloader,
        device,
        num_classes
    ):
        self.model = model
        self.trainloader = trainloader
        self.valloader = valloader
        self.device = device
        self.num_classes = num_classes

    # --------------------------------------------------------
    # RETORNA OS PARÂMETROS LOCAIS
    # --------------------------------------------------------
    def get_parameters(self, config):
        return [val.cpu().numpy() for _, val in self.model.state_dict().items()]

    # --------------------------------------------------------
    # RECEBE OS PESOS DO MODELO GLOBAL (VENDA DO SERVIDOR)
    # --------------------------------------------------------
    def set_parameters(self, parameters):
        params_dict = zip(self.model.state_dict().keys(), parameters)
        state_dict = OrderedDict({k: torch.tensor(v) for k, v in params_dict})
        self.model.load_state_dict(state_dict, strict=True)

    # --------------------------------------------------------
    # FIT — AGORA COM FEDPROX
    # --------------------------------------------------------
    def fit(self, parameters, config):
        # 1. RECEBE PESOS GLOBAIS
        self.set_parameters(parameters)
        self.model.to(self.device)

        # 2. SALVA PESOS GLOBAIS PARA O FEDPROX
        global_params = [p.clone().detach() for p in self.model.parameters()]

        # 3. TREINAMENTO LOCAL
        trained_model = train(
            model=self.model,
            trainloader=self.trainloader,
            valloader=self.valloader,
            device=self.device,
            patience=2,
            global_params=global_params,   # <--
            mu=0.1,               # <--
            verbose=False
        )
        # 4. RETORNA OS PESOS LOCAIS APÓS O TREINO
        return (
            [val.cpu().numpy() for _, val in trained_model.state_dict().items()],
            len(self.trainloader.dataset),
            {}
        )

    # --------------------------------------------------------
    # EVALUATE (sem mudanças)
    # --------------------------------------------------------
    def evaluate(self, parameters, config):
        self.set_parameters(parameters)
        self.model.to(self.device)

        metrics, _ = evaluate(self.model, self.valloader, self.device, num_classes=self.num_classes)
        return float(metrics["loss"]), len(self.valloader.dataset), metrics

    # def evaluate(self, parameters, config):
    #     self.set_parameters(parameters)
    #     self.model.to(self.device)

    #     # Avaliação no TRAIN e não no VAL
    #     metrics, _ = evaluate(
    #         self.model, 
    #         self.trainloader, 
    #         self.device,
    #         num_classes=self.num_classes
    #     )

    #     return float(metrics["loss"]), len(self.trainloader.dataset), metrics




def client_fn(cid: str):
    """Create a Kidney client representing a single organization."""

    model = build_model(num_classes=NUM_CLASSES)
    model = model.to(DEVICE)
    trainloader = trainloaders[int(cid)]
    valloader = valloaders[int(cid)]

    # return KidneyClient(model, trainloader, valloader, DEVICE, NUM_CLASSES).to_client()
    return KidneyClient(model, trainloader, valloader, DEVICE, NUM_CLASSES)


# def client_fn(cid: str):

#     # 1) Cria modelo SEM MOVER PARA GPU
#     model = build_model(num_classes=NUM_CLASSES)
    
#     # 2) NÃO FAÇA model.to(DEVICE) AQUI
#     # Flower vai pegar os pesos no CPU

#     trainloader = trainloaders[int(cid)]
#     valloader = valloaders[int(cid)]

#     return KidneyClient(
#         model=model,
#         trainloader=trainloader,
#         valloader=valloader,
#         device=DEVICE,          # a GPU só será usada dentro do fit()
#         num_classes=NUM_CLASSES
#     )


# def client_fn(cid: str):
#     model = build_model(NUM_CLASSES)

#     # Carrega PESOS DO CPU SEMPRE
#     model.load_state_dict(BASE_STATE)

#     trainloader = trainloaders[int(cid)]
#     valloader = valloaders[int(cid)]

#     return KidneyClient(
#         model=model,
#         trainloader=trainloader,
#         valloader=valloader,
#         device=DEVICE,
#         num_classes=NUM_CLASSES
#     )

## Server

In [ ]:
def server_evaluate_fn(save_path: str = '.'):
    def evaluate_fn(server_round: int, parameters, config):
        model = build_model(NUM_CLASSES)

        set_parameters(model, parameters)
        metrics, cm = evaluate(model, testloader, DEVICE, NUM_CLASSES)
        loss = float(metrics['loss'])

        os.makedirs(save_path, exist_ok=True)

        if server_round != 0:
            torch.save(model.state_dict(), f"{save_path}/model_round{server_round}.pt")

        return loss, metrics

    return evaluate_fn

### Aggregation strategy

- FedAvg, FedAvg, FedAdagrad, FedAdam, FedYogi, Krum, DPFedAvgAdaptive, QFedAvg, FaultTolerantFedAvg

In [ ]:
# (ADJUSTMENTS)
def metrics_agg(metrics):
    # metrics = [(num_examples, {"loss": ..., "accuracy": ...}), ...]
    total_examples = sum(num for num, _ in metrics)
    aggregated = {}
    for key in metrics[0][1].keys():
        aggregated[key] = sum(num * m[key] for num, m in metrics) / total_examples
    return aggregated


# -----------------------------------------------------------------
# CONFIGURAÇÕES ENVIADAS AOS CLIENTES (FedProx precisa disso)
# -----------------------------------------------------------------
def fit_config(server_round: int):
    return {
        "round": server_round,
        "local_epochs": MODEL_CONFIG["epochs"],
    }

def eval_config(server_round: int):
    return {
        "round": server_round
    }


from flwr.server.strategy import FedProx, FedAvg, FedAdagrad, FedAdam, FedYogi, Krum, DPFedAvgAdaptive, QFedAvg, FaultTolerantFedAvg
import flwr as fl

# -----------------------------------------------------------------
# ESTRATÉGIA FEDPROX FINAL (COMPLETA + CORRIGIDA)
# -----------------------------------------------------------------
# strategy = FedProx(
#     fraction_fit=1.0,
#     fraction_evaluate=1.0,
#     min_fit_clients=NUM_CLIENTS,
#     min_evaluate_clients=NUM_CLIENTS,
#     min_available_clients=NUM_CLIENTS,
#     proximal_mu=0.05,
#     evaluate_fn=server_evaluate_fn(save_path='models_25_rounds'),
#     evaluate_metrics_aggregation_fn=metrics_agg
# )

# strategy = FedAvg(
#     fraction_fit=1.0,
#     fraction_evaluate=1.0,
#     min_fit_clients=NUM_CLIENTS,
#     min_evaluate_clients=NUM_CLIENTS,
#     min_available_clients=NUM_CLIENTS,
#     evaluate_fn=server_evaluate_fn(save_path='models_25_rounds'),
#     evaluate_metrics_aggregation_fn=metrics_agg,
# )

strategy = FedAdagrad(
    fraction_fit=1.0,
    fraction_evaluate=1.0,
    min_fit_clients=NUM_CLIENTS,
    min_evaluate_clients=NUM_CLIENTS,
    min_available_clients=NUM_CLIENTS,
    eta=0.1,
    tau=1e-9,
    evaluate_fn=server_evaluate_fn(save_path='models_22_rounds_fedadagrad'),
    evaluate_metrics_aggregation_fn=metrics_agg
)

# strategy = FedAdam(
#     fraction_fit=1.0,
#     fraction_evaluate=1.0,
#     min_fit_clients=NUM_CLIENTS,
#     min_evaluate_clients=NUM_CLIENTS,
#     min_available_clients=NUM_CLIENTS,
#     eta=0.001,         # taxa de aprendizado global
#     eta_l=0.001,       # taxa de aprendizado local
#     beta_1=0.9,
#     beta_2=0.99,
#     tau=1e-9,
#     evaluate_fn=server_evaluate_fn(save_path='models_22_rounds_fedadam'),
#     evaluate_metrics_aggregation_fn=metrics_agg,
# )

# strategy = FedYogi(
#     fraction_fit=1.0,
#     fraction_evaluate=1.0,
#     min_fit_clients=NUM_CLIENTS,
#     min_evaluate_clients=NUM_CLIENTS,
#     min_available_clients=NUM_CLIENTS,
#     eta=0.001,         # taxa de aprendizado global
#     eta_l=0.001,       # taxa de aprendizado local
#     beta_1=0.9,
#     beta_2=0.99,
#     tau=1e-9,
#     evaluate_fn=server_evaluate_fn(save_path='models_22_rounds_fedyogi'),
#     evaluate_metrics_aggregation_fn=metrics_agg,
# )

# strategy = Krum(
#     fraction_fit=1.0,
#     fraction_evaluate=1.0,
#     min_fit_clients=NUM_CLIENTS,
#     min_evaluate_clients=NUM_CLIENTS,
#     min_available_clients=NUM_CLIENTS,
#     krum_config={"multi_krum": False, "num_krum": 1},  # ajustar conforme necessário
#     evaluate_fn=server_evaluate_fn(save_path='models_22_rounds_krum'),
#     evaluate_metrics_aggregation_fn=metrics_agg,
# )

# strategy = DPFedAvgAdaptive(
#     fraction_fit=1.0,
#     fraction_evaluate=1.0,
#     min_fit_clients=NUM_CLIENTS,
#     min_evaluate_clients=NUM_CLIENTS,
#     min_available_clients=NUM_CLIENTS,
#     noise_multiplier=1.0,           # ajustar conforme necessário
#     l2_norm_clip=1.0,               # ajustar conforme necessário
#     adaptive_clip=True,             # ativa o clipping adaptativo
#     evaluate_fn=server_evaluate_fn(save_path='models_22_rounds_dp_fedavg_adaptive'),
#     evaluate_metrics_aggregation_fn=metrics_agg,
# )

# strategy = QFedAvg(
#     fraction_fit=1.0,
#     fraction_evaluate=1.0,
#     min_fit_clients=NUM_CLIENTS,
#     min_evaluate_clients=NUM_CLIENTS,
#     min_available_clients=NUM_CLIENTS,
#     q_param=0.5,  # ajustar conforme desejado
#     evaluate_fn=server_evaluate_fn(save_path='models_22_rounds_qfedavg'),
#     evaluate_metrics_aggregation_fn=metrics_agg,
# )

# strategy = FaultTolerantFedAvg(
#     fraction_fit=1.0,
#     fraction_evaluate=1.0,
#     min_fit_clients=NUM_CLIENTS,
#     min_evaluate_clients=NUM_CLIENTS,
#     min_available_clients=NUM_CLIENTS,
#     evaluate_fn=server_evaluate_fn(save_path='models_22_rounds_fault_toler_fedavg'),
#     evaluate_metrics_aggregation_fn=metrics_agg,
# )

## Simulation

In [ ]:
print(f"running in {DEVICE}")

In [ ]:
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

In [ ]:
history = fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=ROUNDS),
    strategy=strategy,
    client_resources={
        "num_cpus": 1,
        "num_gpus": 0.25
    }
)


### Show Federated learning history

In [ ]:
print("All history: ", history)

In [ ]:
print("Output FL - Loss: ", history.losses_distributed)
print("Output FL - Accuracy: ", history.metrics_distributed)

print("Centralized metrics:", history.metrics_centralized)
print("Centralized losses:", history.losses_centralized)


fl_loss = []
fl_accuracy = []

for loss in history.losses_distributed:
  print(f"Loss in Round {loss[0]}: {loss[1]}")
  fl_loss.append(loss[1])

for accuracy in history.metrics_distributed["accuracy"]:
  print(f"Accuracy in Round {accuracy[0]}: {accuracy[1]}")
  fl_accuracy.append(accuracy[1])

communication_round = range(len(fl_loss))
my_dpi=96
plt.figure(figsize=(800/my_dpi, 600/my_dpi), dpi=my_dpi)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)

plt.plot(communication_round, fl_loss, linewidth=1, linestyle="solid", marker=".", color="black")
plt.xlabel("Communication Round(#)", fontsize=18)
plt.ylabel("Loss", fontsize=18)
plt.grid(linestyle=':', linewidth='0.5')

############################################
communication_round = range(len(fl_accuracy))
my_dpi=96
plt.figure(figsize=(800/my_dpi, 600/my_dpi), dpi=my_dpi)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)

plt.plot(communication_round, fl_accuracy, linewidth=1, linestyle="solid", marker=".", color="black")
plt.xlabel("Communication Round(#)", fontsize=18)
plt.ylabel("Accuracy", fontsize=18)
plt.grid(linestyle=':', linewidth='0.5')

# Server Test set Evaluation

In [ ]:
# (ADJUSTMENTS)
import os
import torch

# ---------------------------------------------------------
# IDENTIFICAR MODELO GLOBAL (MELHOR ROUND)
# ---------------------------------------------------------

model_dir = "models_25_rounds"

# lista arquivos tipo model_roundX.pt
round_files = [f for f in os.listdir(model_dir) if f.startswith("model_round")]

if len(round_files) == 0:
    raise RuntimeError(f"Nenhum modelo encontrado em {model_dir}")

# extrai números dos rounds
round_numbers = sorted([
    int(f.replace("model_round", "").replace(".pt", ""))
    for f in round_files
])

# tenta usar o histórico do Flower para escolher melhor modelo
try:
    # history.losses_distributed = [(round, loss), ...]
    losses = history.losses_distributed
    best_round = min(losses, key=lambda x: x[1])[0]
except:
    # fallback: usa último round
    best_round = round_numbers[-1]

best_model_path = os.path.join(model_dir, f"model_round{best_round}.pt")
print(f"\n🔍 Carregando melhor modelo encontrado: Round {best_round}")
print(f"📁 Path: {best_model_path}")

# ---------------------------------------------------------
# CARREGAR O MODELO GLOBAL
# ---------------------------------------------------------

global_model = build_model(num_classes=NUM_CLASSES)
global_model.load_state_dict(torch.load(best_model_path, map_location=DEVICE))
global_model.to(DEVICE)
global_model.eval()

# ---------------------------------------------------------
# AVALIAÇÃO GLOBAL
# ---------------------------------------------------------

metrics, cm = evaluate(global_model, testloader, DEVICE, NUM_CLASSES)

print("\n📊 Métricas do Melhor Modelo Global:")
print(metrics)


In [ ]:
# (ADJUSTMENTS)
# ----------------------------
# CONFUSION MATRIX (BEST ROUND)
# ----------------------------

labels_map = {0: 'Cyst', 1: 'Normal', 2: 'Stone', 3: 'Tumor'}
labels = [labels_map[i] for i in range(len(labels_map))]

plt.figure(figsize=(7, 6))
sns.heatmap(
    cm,
    annot=True,
    fmt="d",
    cmap="Blues",
    xticklabels=labels,
    yticklabels=labels
)
plt.title(f"Confusion Matrix - Best Round {best_round}")
plt.xlabel("Predicted")
plt.ylabel("True")

# opcional: salva figura para relatório
plt.savefig(f"confusion_matrix_round_{best_round}.png", dpi=200, bbox_inches='tight')

plt.show()



In [ ]:
# ----------------------------
# AVALIAÇÃO CENTRALIZADA FINAL
# ----------------------------

print("\n🔎 Avaliando o melhor modelo global no conjunto de TESTE...\n")

metrics, cm = evaluate(global_model, testloader, DEVICE, NUM_CLASSES)

print("📊 Resultados finais (Centralized Evaluation):")
print(f" - Loss:      {metrics['loss']:.6f}")
print(f" - Accuracy:  {metrics['accuracy']:.6f}")
print(f" - Precision: {metrics['precision']:.6f}")
print(f" - Recall:    {metrics['recall']:.6f}")
print(f" - F1-score:  {metrics['f1-score']:.6f}")
print(f" - AUC:       {metrics['auc']:.6f}")

# salva para logs
centralized_metrics = metrics
